In [1]:
import pandas as pd

In [2]:
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
py.init_notebook_mode(connected=True)

In [3]:
import requests, bs4
url = 'https://www.alltrails.com/parks/us/montana/glacier-national-park'
r = requests.get(url)
soup = bs4.BeautifulSoup(r.text, 'lxml')

In [4]:
ranking = []
link = []
name = []
diff = []
rating = []
location = []
distance = []
elevation = []
route_type =[]
num_reviews = []

In [5]:
for i in range(len(soup.find_all('div', class_="trail-result-card"))):
    
    # ranking
    try:
        ranking.append(soup.find_all('span',class_='item-rank xlate-none')[i].get_text())
    except Exception:
        ranking.append("")
   
    # link
    try:
        url = "https://www.alltrails.com" + (soup.find_all('a',class_='item-link')[i]['href'])
        link.append(url)
    except Exception:
        link.append("")
    
    # trail name
    try:
        name.append(soup.find_all('h3',class_='name xlate-none short')[i].get_text())
    except Exception:
        name.append("")
    
    # difficulty
    try:
        diff.append(soup.find_all('div',class_='difficulty-info')[i].get_text())
    except Exception:
        diff.append("")
    
    # rating
    try:
        rating.append(soup.find_all('div',class_='difficulty-info')[i].find_all('span')[1]['title'])
    except Exception:
        rating.append("")

    # location
    try:
        location.append(soup.find_all('div',class_='clickable xlate-none')[i]['title'])
    except Exception:
        location.append("")
    
    # GRAB TRAIL DETAILS
    r2 = requests.get(url)
    soup2 = bs4.BeautifulSoup(r2.text, 'lxml')
    
    # distance
    try:
        distance.append(soup2.find_all('div', class_="detail-data")[0].get_text())
    except Exception:
        distance.append("")
    
    # elevation gain
    try:
        elevation.append(soup2.find_all('div', class_="detail-data")[1].get_text())
    except Exception:
        elevation.append("")
    
    # route type
    try:
        route_type.append(soup2.find_all('div', class_="detail-data")[2].get_text())
    except Exception:
        route_type.append("")

In [6]:
df = pd.DataFrame(
    {'ranking': ranking,
     'name': name,
     'difficulty': diff,
     'distance_miles': distance,
     'elevation_feet': elevation,
     'route_type': route_type,
     'rating': rating,
     'location': location,
     'link': link
    })

In [7]:
# format columns
df['num_reviews'] = df['difficulty'].str.split("(").str[1].str.strip(")")
df['distance_miles'] = df.distance_miles.str.strip("miles").astype(float)
df['elevation_feet'] = df.elevation_feet.str.replace(",","").str.strip('feet').astype(int)
df['difficulty'] = df.difficulty.str.replace('[^a-zA-Z]','')
df['duration_hours'] = df.distance_miles/2
df['elevation_per_mile'] = df.elevation_feet/df.distance_miles

In [8]:
df = df[['ranking', 'num_reviews', 'name', 'difficulty','duration_hours','elevation_per_mile',
         'distance_miles','elevation_feet','route_type', 'rating', 'location', 'link']]

In [9]:
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val,val)

df_links = df.style.format({'link':make_clickable})

In [10]:
columns = ['num_reviews','duration_hours','elevation_per_mile','distance_miles','elevation_feet','rating']
for col in columns:
    t1 = go.Bar(x= df[df.difficulty == 'EASY']['ranking'],
                y= df[df.difficulty == 'EASY'][col],
                text=df[df.difficulty == 'EASY']['name'],
                textposition = 'auto',
                name='EASY')

    t2 = go.Bar(x= df[df.difficulty == 'MODERATE']['ranking'],
                y= df[df.difficulty == 'MODERATE'][col],
                text=df[df.difficulty == 'MODERATE']['name'],
                textposition = 'auto',
                name='MODERATE')

    t3 = go.Bar(x= df[df.difficulty == 'HARD']['ranking'],
                y= df[df.difficulty == 'HARD'][col],
                text=df[df.difficulty == 'HARD']['name'],
                textposition = 'auto',
                name='HARD')
    data = [t1,t2,t3]
    layout = go.Layout(title=col)
    fig = go.Figure(data,layout)
    py.iplot(fig)

In [11]:
df2

NameError: name 'df2' is not defined